# Day19_4: 오픈소스 LLM 파인튜닝 (PEFT & QLoRA)

## 학습 목표

**Part 1: 기초**
1. 오픈소스 LLM 파인튜닝의 장단점을 상용 API 방식과 비교하여 설명할 수 있습니다.
2. PEFT(Parameter-Efficient Fine-Tuning)의 개념과 필요성을 이해합니다.
3. LoRA(Low-Rank Adaptation)의 핵심 원리를 이해하고 설명할 수 있습니다.
4. QLoRA(Quantized LoRA)가 어떻게 메모리를 절약하는지 이해합니다.
5. Hugging Face 생태계의 주요 라이브러리(transformers, peft, bitsandbytes, trl)를 설명할 수 있습니다.

**Part 2: 심화**
1. 4비트 양자화를 적용하여 Llama 모델을 메모리에 로드할 수 있습니다.
2. 학습 데이터를 Instruct 템플릿에 맞게 포맷팅할 수 있습니다.
3. LoRA 설정을 구성하고 SFTTrainer로 파인튜닝을 실행할 수 있습니다.
4. 학습된 어댑터를 저장하고 모델에 로드하여 추론할 수 있습니다.

---

## 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 오픈소스 LLM | 모델 소유 및 커스터마이징 | Llama, Mistral, Gemma 활용 |
| PEFT/LoRA | 효율적인 파인튜닝 | 소량 파라미터만 학습 (0.1% 미만) |
| QLoRA | 일반 GPU에서 대형 모델 학습 | 8GB VRAM으로 7B 모델 파인튜닝 |
| Hugging Face | 통합 ML 생태계 | 모델, 데이터셋, 학습 도구 |

**분석가 관점**: API 비용 없이 완전히 소유한 맞춤형 LLM을 구축할 수 있습니다. 보안이 중요한 기업 환경이나 오프라인 환경에서 특히 유용하며, 도메인 특화 지식을 모델에 학습시켜 전문 분야 AI 어시스턴트를 만들 수 있습니다!

---

# Part 1: 기초

---

## 1.1 오픈소스 LLM 파인튜닝의 필요성

### 상용 API vs 오픈소스 파인튜닝 비교

| 구분 | 상용 API (OpenAI 등) | 오픈소스 파인튜닝 |
|------|---------------------|------------------|
| **비용** | API 호출당 과금 | 초기 GPU 비용만 |
| **데이터 프라이버시** | 외부 서버로 전송 | 로컬에서 처리 |
| **커스터마이징** | 제한적 | 완전한 자유 |
| **오프라인 사용** | 불가능 | 가능 |
| **모델 소유권** | 없음 | 완전 소유 |
| **학습 난이도** | 쉬움 | 중간~어려움 |

### 오픈소스 LLM을 선택해야 할 때

```
1. 데이터 보안이 중요한 경우 (의료, 금융, 법률)
2. 대량의 추론이 필요하여 API 비용이 부담되는 경우
3. 특수 도메인에 깊이 있는 커스터마이징이 필요한 경우
4. 오프라인/에어갭 환경에서 운영해야 하는 경우
5. 모델을 완전히 통제하고 싶은 경우
```

### 오픈소스 LLM 현황

| 모델 | 개발사 | 파라미터 | 특징 |
|------|--------|----------|------|
| **Llama 3** | Meta | 8B, 70B, 405B | 가장 인기 있는 오픈소스 모델 |
| **Mistral** | Mistral AI | 7B, 8x7B | 효율적인 추론, MOE 아키텍처 |
| **Gemma** | Google | 2B, 7B | 경량화, 안전성 중시 |
| **Qwen** | Alibaba | 0.5B ~ 72B | 다국어 지원 |
| **DeepSeek** | DeepSeek | 7B, 67B | 코딩 특화 |

> **주의**: 대부분의 오픈소스 모델은 라이선스 조건이 있습니다. 상업적 사용 전 반드시 확인하세요!

---

## 1.2 파인튜닝의 메모리 문제와 PEFT

### 전체 파인튜닝(Full Fine-Tuning)의 한계

```
LLM 파인튜닝에 필요한 메모리 (추정):

모델 가중치 (FP16): 파라미터 수 x 2 bytes
그래디언트:        파라미터 수 x 2 bytes  
옵티마이저 상태:    파라미터 수 x 8 bytes (Adam)
활성화:            배치 크기에 따라 가변

예시: 7B 파라미터 모델
- 가중치: 7B x 2 = 14GB
- 그래디언트: 14GB
- 옵티마이저: 56GB
- 총 필요: 약 84GB+ VRAM

-> 일반 GPU로는 불가능!
```

### PEFT (Parameter-Efficient Fine-Tuning)

**핵심 아이디어**: 전체 모델을 학습하지 않고, 아주 작은 부분만 학습하자!

| PEFT 기법 | 핵심 아이디어 | 학습 파라미터 비율 |
|-----------|--------------|------------------|
| **LoRA** | 저차원 행렬 분해 | 0.1% ~ 1% |
| **Prefix Tuning** | 학습 가능한 프롬프트 추가 | < 1% |
| **Adapter** | 작은 네트워크 삽입 | 1% ~ 5% |
| **IA3** | 활성화 스케일링 | < 0.1% |

---

## 1.3 LoRA (Low-Rank Adaptation)

### LoRA의 핵심 원리

```
기존 가중치 행렬: W (d x k)
LoRA 분해: W + BA
  - B: (d x r) 행렬
  - A: (r x k) 행렬
  - r: rank (보통 8, 16, 32 등 작은 값)

원래 파라미터: d x k
LoRA 파라미터: d x r + r x k = r(d + k)

예시: d=4096, k=4096, r=16
  원래: 4096 x 4096 = 16,777,216
  LoRA: 16 x (4096 + 4096) = 131,072
  -> 약 0.78% 만 학습!
```

### LoRA 아키텍처

```mermaid
flowchart LR
    Input["Input\n(batch, seq, d)"] --> Frozen["Pretrained Weights W\n(고정, 학습 안함)"]
    Input --> LoRA_A["LoRA A\n(d x r)"]
    LoRA_A --> LoRA_B["LoRA B\n(r x k)"]
    Frozen --> Add((+))
    LoRA_B --> Scale["x alpha/r"]
    Scale --> Add
    Add --> Output["Output\n(batch, seq, k)"]
    
    style Frozen fill:#ffffff,color:#000000
    style LoRA_A fill:#e3f2fd,color:#000000
    style LoRA_B fill:#e3f2fd,color:#000000
    style Scale fill:#e3f2fd,color:#000000
```

### LoRA의 장점

| 장점 | 설명 |
|------|------|
| **메모리 효율성** | 전체의 0.1~1% 파라미터만 학습 |
| **학습 속도** | 학습해야 할 파라미터가 적어 빠름 |
| **어댑터 교환** | 기본 모델은 그대로, 어댑터만 교체하여 다양한 태스크 수행 |
| **성능 유지** | 전체 파인튜닝과 유사한 성능 |
| **저장 효율** | 어댑터 파일 크기 수십 MB (모델 전체는 수십 GB) |

---

## 1.4 QLoRA (Quantized LoRA)

### 양자화(Quantization)란?

```
정밀도에 따른 메모리 사용량:

FP32 (32비트): 파라미터당 4 bytes
FP16 (16비트): 파라미터당 2 bytes
INT8 (8비트):  파라미터당 1 byte
INT4 (4비트):  파라미터당 0.5 bytes

7B 모델 메모리 (가중치만):
- FP32: 28GB
- FP16: 14GB
- INT8: 7GB
- INT4: 3.5GB  <- QLoRA!
```

### QLoRA의 혁신

QLoRA = 4비트 양자화 + LoRA

| 구성요소 | 설명 |
|----------|------|
| **4-bit NormalFloat (NF4)** | 정규분포에 최적화된 4비트 데이터 타입 |
| **Double Quantization** | 양자화 상수도 양자화하여 추가 메모리 절약 |
| **Paged Optimizers** | GPU 메모리 초과 시 CPU로 페이징 |

```
QLoRA의 메모리 절약 효과:

7B 모델 파인튜닝:
- 전체 FT (FP16): 84GB+ VRAM
- LoRA (FP16): 14GB+ VRAM
- QLoRA (4-bit): 6GB+ VRAM  <- 일반 GPU 가능!
```

---

## 1.5 Hugging Face 라이브러리 생태계

### 주요 라이브러리

| 라이브러리 | 역할 | 주요 기능 |
|-----------|------|----------|
| **transformers** | 모델 & 토크나이저 | 모델 로드, 추론, 저장 |
| **peft** | PEFT 기법 구현 | LoRA, Prefix Tuning 등 |
| **bitsandbytes** | 양자화 | 4-bit, 8-bit 양자화 |
| **trl** | 강화학습 기반 학습 | SFTTrainer, PPO, DPO |
| **datasets** | 데이터셋 관리 | 로드, 전처리, 스트리밍 |
| **accelerate** | 분산 학습 | multi-GPU, mixed precision |

In [ ]:
# 필요한 라이브러리 설치 (Colab 또는 새 환경에서 실행)
# GPU 환경 필요! (CPU에서는 양자화가 제한됨)

# !pip install transformers accelerate peft bitsandbytes datasets trl -q

In [ ]:
import torch
import numpy as np
import pandas as pd

# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"PyTorch 버전: {torch.__version__}")
print(f"사용 가능한 디바이스: {device}")

if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("\n주의: GPU가 없습니다. QLoRA 실습에는 GPU가 권장됩니다.")

In [ ]:
# Hugging Face Hub 로그인 (모델 다운로드에 필요할 수 있음)
# Llama 모델은 Meta 승인이 필요합니다!

# from huggingface_hub import login
# from dotenv import load_dotenv
# import os

# load_dotenv()
# login(token=os.getenv("HUGGINGFACE_TOKEN"))

---

# Part 2: 심화

---

## 2.1 모델 로드 및 양자화 설정

### BitsAndBytes 양자화 설정

```python
BitsAndBytesConfig 주요 옵션:

load_in_4bit=True        # 4비트 양자화 활성화
bnb_4bit_quant_type="nf4"  # NormalFloat4 (권장)
bnb_4bit_compute_dtype=torch.float16  # 연산 정밀도
bnb_4bit_use_double_quant=True  # 이중 양자화 (추가 메모리 절약)
```

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델 ID (Hugging Face Hub에서 선택)
# Llama 3.2 1B: 가장 가벼운 Llama 모델
model_id = "meta-llama/Llama-3.2-1B-Instruct"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Llama 모델은 pad_token이 없으므로 설정
tokenizer.pad_token = tokenizer.eos_token

print(f"토크나이저 로드 완료")
print(f"Vocab Size: {tokenizer.vocab_size}")
print(f"Model Max Length: {tokenizer.model_max_length}")

In [ ]:
# 모델 로드 (환경에 따라 양자화 적용)

if device == "cuda":
    # GPU 환경: 4비트 양자화 적용
    from transformers import BitsAndBytesConfig
    
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,                    # 4비트 양자화
        bnb_4bit_quant_type="nf4",            # NormalFloat4 타입
        bnb_4bit_compute_dtype=torch.float16, # 연산은 FP16
        bnb_4bit_use_double_quant=False       # 더블 양자화 (선택)
    )
    
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto"
    )
    print("GPU 환경: 4비트 양자화 모델 로드 완료")
else:
    # CPU 환경: 양자화 없이 로드
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float32,
        device_map="auto"
    )
    print("CPU 환경: 양자화 없이 모델 로드 완료")

print(f"\n모델 타입: {type(model).__name__}")

In [ ]:
# 모델 정보 확인
def count_parameters(model):
    """모델의 파라미터 수 계산"""
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable

total, trainable = count_parameters(model)
print(f"총 파라미터: {total:,}")
print(f"학습 가능 파라미터: {trainable:,}")
print(f"학습 비율: {trainable/total*100:.2f}%")

---

## 2.2 데이터셋 준비 및 포맷팅

### Instruct 모델의 프롬프트 템플릿

각 모델은 고유한 프롬프트 형식을 사용합니다.

```
Llama 3 Instruct 템플릿:

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system_message}<|eot_id|><|start_header_id|>user<|end_header_id|>

{user_message}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{assistant_message}<|eot_id|>
```

In [ ]:
from datasets import Dataset

# 파인튜닝을 위한 데이터셋 (데이터 분석 Q&A)
qa_data = [
    {
        "question": "데이터 분석에서 EDA(탐색적 데이터 분석)의 목적은 무엇인가요?",
        "answer": "EDA는 데이터의 패턴, 이상치, 분포, 관계성을 파악하여 데이터의 특성을 이해하고, 이후 분석 방향을 결정하는 데 도움을 주는 과정입니다. 시각화와 기술통계를 통해 데이터의 품질과 특성을 종합적으로 평가합니다."
    },
    {
        "question": "파이썬 pandas에서 결측치를 처리하는 방법에는 어떤 것들이 있나요?",
        "answer": "결측치 처리 방법으로는 1) dropna()로 결측치가 포함된 행/열 삭제, 2) fillna()로 평균, 중앙값, 최빈값 등으로 대체, 3) forward fill/backward fill로 이전/이후 값으로 채우기, 4) 보간법(interpolation) 사용 등이 있습니다."
    },
    {
        "question": "머신러닝에서 과적합(Overfitting)을 방지하는 방법은 무엇인가요?",
        "answer": "과적합 방지 방법으로는 1) 교차검증(Cross-validation) 사용, 2) 정규화(Regularization) 적용, 3) 드롭아웃(Dropout) 사용, 4) 데이터 증강(Data Augmentation), 5) 앙상블 방법 사용, 6) 조기 종료(Early Stopping) 등이 있습니다."
    },
    {
        "question": "SQL에서 JOIN의 종류와 차이점을 설명해주세요.",
        "answer": "SQL JOIN의 주요 종류는 1) INNER JOIN: 양쪽 테이블에 모두 존재하는 데이터만, 2) LEFT JOIN: 왼쪽 테이블의 모든 데이터와 매칭되는 오른쪽 데이터, 3) RIGHT JOIN: 오른쪽 테이블의 모든 데이터와 매칭되는 왼쪽 데이터, 4) FULL OUTER JOIN: 양쪽 테이블의 모든 데이터를 포함합니다."
    },
    {
        "question": "통계에서 p-value의 의미와 해석 방법은 무엇인가요?",
        "answer": "p-value는 귀무가설이 참일 때 관찰된 결과보다 더 극단적인 결과가 나올 확률입니다. 일반적으로 p-value < 0.05일 때 통계적으로 유의하다고 판단하며, 이는 5% 유의수준에서 귀무가설을 기각할 수 있다는 의미입니다."
    },
    {
        "question": "데이터 전처리에서 정규화(Normalization)와 표준화(Standardization)의 차이는?",
        "answer": "정규화는 데이터를 0~1 범위로 변환하는 Min-Max Scaling을 의미하며, 이상치에 민감합니다. 표준화는 평균을 0, 표준편차를 1로 만드는 Z-score 변환으로, 이상치에 덜 민감하며 정규분포를 가정하는 알고리즘에 적합합니다."
    },
    {
        "question": "시계열 분석에서 계절성(Seasonality)과 추세(Trend)의 차이는?",
        "answer": "추세는 장기적인 증가/감소 패턴을 의미하며, 시간에 따른 일관된 방향성을 가집니다. 계절성은 일정한 주기로 반복되는 패턴으로, 월별, 계절별, 요일별 등 규칙적인 변동을 나타냅니다."
    },
    {
        "question": "딥러닝에서 배치 정규화(Batch Normalization)의 효과는?",
        "answer": "배치 정규화는 1) 학습 속도 향상, 2) 가중치 초기화 민감도 감소, 3) 더 높은 학습률 사용 가능, 4) 정규화 효과로 드롭아웃 필요성 감소, 5) 내부 공변량 변화(Internal Covariate Shift) 완화 등의 효과가 있습니다."
    },
    {
        "question": "자연어 처리에서 임베딩(Embedding)이란 무엇인가요?",
        "answer": "임베딩은 단어나 문장을 고정 크기의 밀집 벡터로 변환하는 것입니다. 비슷한 의미의 단어는 벡터 공간에서 가까이 위치합니다. Word2Vec, GloVe, FastText 등이 대표적인 단어 임베딩 방법이며, BERT, GPT 등은 문맥을 고려한 동적 임베딩을 생성합니다."
    },
    {
        "question": "A/B 테스트에서 통계적 검정력(Statistical Power)이 중요한 이유는?",
        "answer": "통계적 검정력은 실제로 차이가 있을 때 이를 올바르게 감지할 확률입니다. 검정력이 낮으면 실제 효과를 놓칠 위험이 있어, 적절한 표본 크기와 실험 설계가 필요합니다. 일반적으로 80% 이상의 검정력을 목표로 합니다."
    }
]

dataset = Dataset.from_list(qa_data)
print(f"데이터셋 크기: {len(dataset)}")
print(f"\n샘플 데이터:")
print(f"Q: {dataset[0]['question']}")
print(f"A: {dataset[0]['answer'][:100]}...")

In [ ]:
# Llama 3 Instruct 템플릿에 맞게 데이터 포맷팅
def format_llama3_prompt(example):
    """Llama 3 Instruct 템플릿으로 변환"""
    prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{example['question']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{example['answer']}<|eot_id|>"""
    return {"text": prompt}

# 데이터셋 포맷팅
formatted_dataset = dataset.map(format_llama3_prompt)

print("포맷팅된 첫 번째 데이터 예시:")
print("="*60)
print(formatted_dataset[0]['text'])

---

## 2.3 LoRA 설정 및 모델 준비

### LoRA 하이퍼파라미터

| 파라미터 | 설명 | 권장 값 |
|---------|------|--------|
| **r (rank)** | LoRA 행렬의 랭크 | 8, 16, 32, 64 |
| **lora_alpha** | 스케일링 계수 | r의 2배 |
| **lora_dropout** | 드롭아웃 비율 | 0.05 ~ 0.1 |
| **target_modules** | LoRA 적용할 레이어 | q_proj, k_proj, v_proj, o_proj |
| **task_type** | 태스크 유형 | CAUSAL_LM |

In [ ]:
from peft import LoraConfig, get_peft_model

# LoRA 설정
lora_config = LoraConfig(
    r=16,                      # LoRA rank (행렬 분해 차원)
    lora_alpha=32,             # LoRA alpha (스케일링 팩터)
    lora_dropout=0.05,         # 드롭아웃 비율
    bias="none",               # bias 학습 여부
    task_type="CAUSAL_LM",     # 태스크 유형
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]  # Llama Attention 레이어
)

print("LoRA 설정:")
print(f"  - Rank (r): {lora_config.r}")
print(f"  - Alpha: {lora_config.lora_alpha}")
print(f"  - Dropout: {lora_config.lora_dropout}")
print(f"  - Target Modules: {lora_config.target_modules}")

In [ ]:
# LoRA 적용 후 파라미터 확인
peft_model = get_peft_model(model, lora_config)

# 학습 가능한 파라미터 출력
peft_model.print_trainable_parameters()

---

## 2.4 학습 설정 및 SFTTrainer

### SFTTrainer (Supervised Fine-tuning Trainer)

TRL 라이브러리의 SFTTrainer는 LLM 파인튜닝에 최적화된 트레이너입니다.

```
SFTTrainer의 장점:
1. 간결한 코드로 파인튜닝 가능
2. PEFT 통합 지원
3. 자동 토큰화 및 배치 처리
4. 메모리 최적화 기능
```

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

# 학습 설정
training_args = TrainingArguments(
    output_dir="./llama3-finetuned",      # 결과 저장 경로
    per_device_train_batch_size=1,        # 배치 크기 (메모리에 따라 조정)
    gradient_accumulation_steps=4,        # 그래디언트 누적 (효과적 배치=4)
    learning_rate=2e-4,                   # 학습률
    num_train_epochs=3,                   # 에포크 수
    logging_steps=1,                      # 로깅 주기
    save_total_limit=2,                   # 체크포인트 저장 개수
    fp16=(device == "cuda"),              # GPU면 FP16 사용
    optim="adamw_torch",                  # 옵티마이저
    warmup_ratio=0.1,                     # Warmup 비율
)

print("학습 설정 완료:")
print(f"  - 배치 크기: {training_args.per_device_train_batch_size}")
print(f"  - 그래디언트 누적: {training_args.gradient_accumulation_steps}")
print(f"  - 학습률: {training_args.learning_rate}")
print(f"  - 에포크: {training_args.num_train_epochs}")

In [ ]:
# SFTTrainer 초기화
trainer = SFTTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=formatted_dataset,
    peft_config=lora_config,
    tokenizer=tokenizer,
)

print("SFTTrainer 준비 완료!")

---

## 2.5 학습 실행 및 모델 저장

### 주의사항

- GPU 환경에서 실행을 권장합니다
- CPU 환경에서는 학습이 매우 느립니다
- 메모리 부족 시 배치 크기를 줄이세요

In [ ]:
# 학습 실행
print("파인튜닝을 시작합니다...")
print("="*60)

# 실제 학습 (시간이 오래 걸릴 수 있음)
# trainer.train()

# 데모용: 학습 건너뛰기 (이미 학습된 어댑터 사용)
print("[데모] 학습 과정은 GPU 환경에서 실행하세요.")
print("실제 학습: trainer.train()")

In [ ]:
# 어댑터 저장
adapter_path = "./models/llama3-qa-adapter"

# 학습 후 저장
# trainer.save_model(adapter_path)
# print(f"어댑터가 '{adapter_path}'에 저장되었습니다.")

print("[데모] 어댑터 저장:")
print(f"trainer.save_model('{adapter_path}')")

---

## 2.6 파인튜닝된 모델로 추론

### 어댑터 로드 및 추론

저장된 어댑터를 기본 모델에 로드하여 추론을 수행합니다.

In [ ]:
# 추론 함수 정의
def generate_response(model, tokenizer, question, max_new_tokens=200):
    """
    파인튜닝된 모델로 답변 생성
    
    Parameters:
    - model: 파인튜닝된 모델
    - tokenizer: 토크나이저
    - question: 질문 텍스트
    - max_new_tokens: 최대 생성 토큰 수
    
    Returns:
    - 생성된 답변 텍스트
    """
    # Llama 3 프롬프트 형식
    prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
    
    # 토큰화
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # 생성
    model.eval()
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # 디코딩
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 프롬프트 부분 제거하고 답변만 추출
    if "assistant" in response.lower():
        answer = response.split("assistant")[-1].strip()
    else:
        answer = response
    
    return answer

print("추론 함수 정의 완료")

In [ ]:
# 테스트 질문
test_questions = [
    "A/B 테스트에서 가장 중요한 것은 무엇인가요?",
    "데이터 분석가가 갖춰야 할 핵심 역량은?",
    "머신러닝 모델의 성능을 평가하는 방법을 알려주세요."
]

# 추론 실행 (파인튜닝 전 모델)
print("파인튜닝 전 모델 응답:")
print("="*60)

for q in test_questions:
    print(f"\nQ: {q}")
    try:
        answer = generate_response(model, tokenizer, q, max_new_tokens=100)
        print(f"A: {answer}")
    except Exception as e:
        print(f"Error: {e}")
    print("-"*40)

### 어댑터 로드 (학습 완료 후)

```python
from peft import PeftModel

# 저장된 어댑터 로드
finetuned_model = PeftModel.from_pretrained(
    model,           # 기본 모델
    adapter_path     # 어댑터 경로
)

# 파인튜닝된 모델로 추론
answer = generate_response(finetuned_model, tokenizer, "질문")
```

---

## 실습 퀴즈

**난이도 분포**: 기본 3개 / 응용 3개 / 복합 2개 / 종합 2개

---

### Q1. PEFT 개념 이해 (기본)

**문제**: 다음 빈칸을 채우세요.

1. PEFT는 (______)의 약자로, 전체 모델이 아닌 아주 작은 부분만 학습합니다.
2. LoRA는 (______)을 통해 원본 가중치 행렬을 근사합니다.
3. QLoRA는 (______)비트 양자화와 LoRA를 결합한 기술입니다.

In [ ]:
# 여기에 답을 작성하세요
answer1 = ""
answer2 = ""
answer3 = ""

print(f"1. {answer1}")
print(f"2. {answer2}")
print(f"3. {answer3}")

### Q2. LoRA 파라미터 비율 계산 (기본)

**문제**: 다음 조건에서 LoRA 학습 파라미터 비율을 계산하세요.

- 원래 가중치 행렬: 4096 x 4096
- LoRA rank (r): 8

LoRA 파라미터 수와 원래 파라미터 대비 비율을 계산하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q3. 양자화 메모리 계산 (기본)

**문제**: 7B 파라미터 모델의 메모리 사용량을 비교하세요.

1. FP32 (32비트)
2. FP16 (16비트)
3. INT4 (4비트)

각각 몇 GB가 필요한지 계산하세요. (가중치만)

In [ ]:
# 여기에 코드를 작성하세요


### Q4. 토크나이저 사용 (응용)

**문제**: 아래 텍스트를 Llama 3 Instruct 템플릿으로 변환하세요.

- 시스템 메시지: "당신은 데이터 분석 전문가입니다."
- 사용자 질문: "pandas와 numpy의 차이점은?"

Llama 3 템플릿 형식을 사용하세요.

In [ ]:
# 여기에 코드를 작성하세요


### Q5. LoRA 설정 변경 (응용)

**문제**: 다음 요구사항에 맞는 LoraConfig를 작성하세요.

- rank: 32
- alpha: 64
- dropout: 0.1
- Llama의 모든 Linear 레이어에 적용 (q_proj, k_proj, v_proj, o_proj, gate_proj, up_proj, down_proj)

In [ ]:
from peft import LoraConfig

# 여기에 코드를 작성하세요


### Q6. 학습 인자 최적화 (응용)

**문제**: 다음 상황에 맞는 TrainingArguments를 작성하세요.

- 목표: 메모리 제한(8GB VRAM)에서 안정적인 학습
- 데이터셋: 1000개 샘플
- 에포크: 2

배치 크기, 그래디언트 누적, 학습률을 적절히 설정하세요.

In [ ]:
from transformers import TrainingArguments

# 여기에 코드를 작성하세요


### Q7. 데이터셋 포맷팅 함수 (복합)

**문제**: 다양한 형식의 데이터를 Llama 3 템플릿으로 변환하는 함수를 작성하세요.

지원해야 할 입력 형식:
1. `{"question": "...", "answer": "..."}` (Q&A)
2. `{"instruction": "...", "input": "...", "output": "..."}` (Alpaca 형식)
3. `{"prompt": "...", "response": "..."}` (일반 형식)

In [ ]:
def format_to_llama3(example):
    """
    다양한 형식의 데이터를 Llama 3 템플릿으로 변환
    """
    # 여기에 코드를 작성하세요
    pass

# 테스트
test_examples = [
    {"question": "머신러닝이란?", "answer": "데이터로부터 학습하는 AI입니다."},
    {"instruction": "번역하세요", "input": "Hello", "output": "안녕하세요"},
    {"prompt": "파이썬의 장점", "response": "읽기 쉽고 배우기 쉽습니다."}
]

for ex in test_examples:
    print(format_to_llama3(ex))
    print("-"*40)

### Q8. 추론 함수 개선 (복합)

**문제**: 생성 파라미터를 조절할 수 있는 개선된 추론 함수를 작성하세요.

요구사항:
1. temperature, top_p, top_k 파라미터 지원
2. 시스템 프롬프트 지원
3. 생성 결과와 함께 토큰 수 반환

In [ ]:
def generate_with_params(
    model, 
    tokenizer, 
    question,
    system_prompt=None,
    temperature=0.7,
    top_p=0.9,
    top_k=50,
    max_new_tokens=200
):
    """
    파라미터 조절 가능한 추론 함수
    
    Returns:
    - dict: {'answer': str, 'input_tokens': int, 'output_tokens': int}
    """
    # 여기에 코드를 작성하세요
    pass

# 테스트
# result = generate_with_params(
#     model, tokenizer, 
#     "파이썬의 장점은?",
#     system_prompt="간결하게 답변하세요.",
#     temperature=0.3
# )
# print(result)

### Q9. 전체 파인튜닝 파이프라인 (종합)

**문제**: 주어진 CSV 파일에서 데이터를 로드하고 파인튜닝하는 전체 파이프라인을 작성하세요.

단계:
1. CSV 로드 (question, answer 컬럼)
2. 데이터셋 변환
3. 템플릿 포맷팅
4. LoRA 설정
5. 학습 인자 설정
6. SFTTrainer 구성

In [ ]:
def finetune_from_csv(csv_path, model_id, output_dir):
    """
    CSV 데이터로 LLM 파인튜닝 파이프라인
    
    Parameters:
    - csv_path: Q&A CSV 파일 경로
    - model_id: Hugging Face 모델 ID
    - output_dir: 결과 저장 경로
    
    Returns:
    - trainer: 구성된 SFTTrainer
    """
    # 여기에 코드를 작성하세요
    pass

# 사용 예시
# trainer = finetune_from_csv(
#     "data/qa_dataset.csv",
#     "meta-llama/Llama-3.2-1B-Instruct",
#     "./output"
# )
# trainer.train()

### Q10. 모델 비교 평가 (종합)

**문제**: 파인튜닝 전후 모델의 응답을 비교 평가하는 함수를 작성하세요.

요구사항:
1. 여러 테스트 질문에 대해 양쪽 모델의 응답 생성
2. 결과를 DataFrame으로 정리
3. 응답 길이, 키워드 포함 여부 등 기본 메트릭 계산

In [ ]:
import pandas as pd

def compare_models(base_model, finetuned_model, tokenizer, test_questions, keywords=None):
    """
    파인튜닝 전후 모델 비교
    
    Parameters:
    - base_model: 파인튜닝 전 모델
    - finetuned_model: 파인튜닝 후 모델  
    - tokenizer: 토크나이저
    - test_questions: 테스트 질문 리스트
    - keywords: 확인할 키워드 딕셔너리 {질문: [키워드들]}
    
    Returns:
    - pd.DataFrame: 비교 결과
    """
    # 여기에 코드를 작성하세요
    pass

# 테스트
test_questions = [
    "EDA의 목적은 무엇인가요?",
    "과적합을 방지하는 방법은?",
    "p-value란 무엇인가요?"
]

# result_df = compare_models(model, finetuned_model, tokenizer, test_questions)
# print(result_df)

---

## 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 내용 | 실무 활용 |
|------|----------|----------|
| 오픈소스 LLM | 완전한 모델 소유, API 비용 없음 | Llama, Mistral, Gemma |
| PEFT | 전체의 1% 미만만 학습 | 효율적 파인튜닝 |
| LoRA | 저차원 행렬 분해 (BA) | 어댑터 방식 학습 |
| QLoRA | 4비트 양자화 + LoRA | 일반 GPU에서 학습 |
| Hugging Face | 통합 ML 생태계 | transformers, peft, trl |

### Part 2: 심화 핵심 요약

| 개념 | 핵심 기법 | 언제 사용? |
|------|----------|----------|
| BitsAndBytesConfig | NF4, 이중 양자화 | GPU 메모리 제한 시 |
| 프롬프트 템플릿 | Instruct 형식 준수 | 모델별 형식 맞춤 |
| LoraConfig | r, alpha, target_modules | 파인튜닝 설정 |
| SFTTrainer | 간결한 학습 코드 | LLM 파인튜닝 |

### LoRA 하이퍼파라미터 가이드

```
r (rank):       8~64, 높을수록 표현력 증가/메모리 증가
lora_alpha:     r의 2배 권장
lora_dropout:   0.05~0.1
target_modules: Attention (q,k,v,o) + FFN (gate,up,down)
```

### 실무 팁

1. **모델 선택**: 태스크에 맞는 크기의 모델 선택 (1B~70B)
2. **데이터 품질**: 고품질 데이터 100개 > 저품질 데이터 10000개
3. **템플릿 준수**: 각 모델의 Instruct 템플릿 정확히 따르기
4. **학습률**: 2e-4 ~ 2e-5 (QLoRA는 높게 가능)
5. **에포크**: 1~3회 (과적합 주의)
6. **평가**: 학습 데이터에 없는 질문으로 테스트